In [5]:
# Step 3: Define Custom Dataset and DataLoader
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
import torch

# Load and split data
# DATASET_FOLDER = '/content/drive/MyDrive/temp_dataset'
IMAGE_FOLDER = 'images/train'
df = pd.read_csv("dataset/merged_train_with_image.csv")
print(f"Loaded {len(df)} rows")
print("Columns in temp_train.csv:", df.columns.tolist())
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print(f"Train: {len(train_df)} rows, Test: {len(test_df)} rows")

def custom_collate_fn(batch):
    """
    Custom collate function to handle PIL Images, strings, and prices.
    Returns images as a list, texts as a list, and prices as a tensor.
    """
    images, texts, prices = zip(*batch)
    return list(images), list(texts), torch.tensor(prices, dtype=torch.float32)

class ProductDataset(Dataset):
    def __init__(self, df, image_folder):
        """
        Initialize dataset with DataFrame and image folder.
        - df: Contains [sample_id, product_name, catalog_content_clean, price, value, unit, image]
        - image_folder: Directory with images named in 'image' column (e.g., 33127.jpg)
        """
        self.df = df
        self.image_folder = image_folder
        self.transform = None  # ViTImageProcessor handles preprocessing

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_folder, row['image'])  # Use 'image' column
        try:
            img = Image.open(image_path).convert('RGB')
        except FileNotFoundError:
            print(f"Image not found at {image_path}, using placeholder.")
            img = Image.new('RGB', (224, 224), (0, 0, 0))
        text = row['catalog_content_clean']
        price = float(row['price'])
        return img, text, price

# Create and test DataLoaders
try:
    train_dataset = ProductDataset(train_df, IMAGE_FOLDER)
    test_dataset = ProductDataset(test_df, IMAGE_FOLDER)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0, pin_memory=True, collate_fn=custom_collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=16, num_workers=0, pin_memory=True, collate_fn=custom_collate_fn)
    print("DataLoaders ready with your dataset structure.")
    # Test one batch
    for batch_images, batch_texts, batch_prices in train_loader:
        print(f"Batch loaded: {len(batch_images)} images, {len(batch_texts)} texts, {batch_prices.shape} prices")
        print(f"Image type: {type(batch_images[0])}, Price type: {type(batch_prices)}")
        break
except Exception as e:
    print(f"Error creating DataLoaders: {e}")

Loaded 74999 rows
Columns in temp_train.csv: ['sample_id', 'product_name', 'catalog_content_clean', 'price', 'value', 'unit', 'image']
Train: 59999 rows, Test: 15000 rows
DataLoaders ready with your dataset structure.
Batch loaded: 16 images, 16 texts, torch.Size([16]) prices
Image type: <class 'PIL.Image.Image'>, Price type: <class 'torch.Tensor'>


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.0 MB 8.3 MB/s eta 0:00:02
   ----------- ---------------------------- 3.4/12.0 MB 9.1 MB/s eta 0:00:01
   ------------------- -------------------- 5.8/12.0 MB 10.1 MB/s eta 0:00:01
   --------------------------- ------------ 8.4/12.0 MB 10.6 MB/s eta 0:00:01
   ----------------------------------- ---- 10.7/12.0 MB 10.8 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 11.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/564.3 kB ? eta -:--:--
   ---------------------------------------- 564.3/564.3 kB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ----------------------------------- ---- 2.4/2.7 MB 12.2 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 11.9 MB/s eta 0:00:00


In [3]:
# Step 4: Define Multimodal Model
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import ViTImageProcessor, ViTModel, BertTokenizer, BertModel, DistilBertModel

class MultimodalModel(nn.Module):
    def __init__(self, fine_tune_encoders=True):
        super(MultimodalModel, self).__init__()
        self.image_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
        self.image_model = ViTModel.from_pretrained('google/vit-base-patch16-224')
        self.text_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.text_model = BertModel.from_pretrained('bert-base-uncased')
        self.text_proj = nn.Linear(768, 512)
        self.image_proj = nn.Linear(768, 512)
        # Add projection layer to reduce 1024-dim (512+512) to 768-dim for DistilBERT
        self.concat_proj = nn.Linear(1024, 768)
        self.transformer = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.regression = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.ReLU()
        )
        if not fine_tune_encoders:
            for param in self.image_model.parameters():
                param.requires_grad = False
            for param in self.text_model.parameters():
                param.requires_grad = False
        print("Encoders fine-tuning:", fine_tune_encoders)

    def forward(self, images, texts):
        # Image encoding
        inputs = self.image_processor(images=images, return_tensors="pt").to(device)
        image_outputs = self.image_model(**inputs).last_hidden_state.mean(dim=1)
        image_proj = F.normalize(self.image_proj(image_outputs), dim=-1)
        # Text encoding
        inputs = self.text_tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        text_outputs = self.text_model(**inputs).last_hidden_state.mean(dim=1)
        text_proj = F.normalize(self.text_proj(text_outputs), dim=-1)
        # Concatenate and project to 768-dim
        concat_emb = torch.cat([text_proj, image_proj], dim=-1)  # [batch_size, 1024]
        concat_emb = self.concat_proj(concat_emb).unsqueeze(1)  # [batch_size, 1, 768]
        # Transformer
        transformer_out = self.transformer(inputs_embeds=concat_emb).last_hidden_state.mean(dim=1)
        # Regression
        price = self.regression(transformer_out)
        return price.squeeze(), text_proj, image_proj
device = torch.device('cuda')  # Define device
model = MultimodalModel(fine_tune_encoders=True).to(device)
print("Model initialized.")

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Encoders fine-tuning: True
Model initialized.


In [ ]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import time
import os  # Add this import for os.path operations
from tqdm import tqdm

# --- Define Losses ---
def contrastive_loss(text_proj, image_proj, temperature=0.07):
    """
    Contrastive loss to align image and text projections.
    """
    similarity = text_proj @ image_proj.T / temperature
    labels = torch.arange(text_proj.size(0)).to(device)
    return F.cross_entropy(similarity, labels)

def smape(y_true, y_pred):
    """
    Symmetric Mean Absolute Percentage Error for evaluation.
    """
    y_true = y_true.cpu().numpy() if isinstance(y_true, torch.Tensor) else np.array(y_true)
    y_pred = y_pred.cpu().numpy() if isinstance(y_pred, torch.Tensor) else np.array(y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_pred - y_true) / np.maximum(denominator, 1e-8)  # avoid /0
    return np.mean(diff) * 100

# --- Optimizer and Loss ---
criterion = torch.nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

start_time = time.time()
num_epochs = 100

for epoch in range(num_epochs):
    # ------------------- TRAINING -------------------
    model.train()
    train_mse, train_cont = 0, 0
    train_preds, train_true = [], []

    print(f"\n🔹 Epoch {epoch + 1}/{num_epochs} - Training")
    for batch_images, batch_texts, batch_prices in tqdm(train_loader, desc="Training Progress", leave=False):
        optimizer.zero_grad()
        pred, text_proj, image_proj = model(batch_images, batch_texts)

        mse = criterion(pred, batch_prices.to(device))
        cont = contrastive_loss(text_proj, image_proj)

        loss = mse + 0.5 * cont
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        train_mse += mse.item()
        train_cont += cont.item()
        train_preds.extend(pred.cpu().detach().numpy())
        train_true.extend(batch_prices.cpu().numpy())

    avg_train_mse = train_mse / len(train_loader)
    avg_train_cont = train_cont / len(train_loader)
    train_smape = smape(np.array(train_true), np.array(train_preds))
    print(f"✅ Train MSE = {avg_train_mse:.4f}, Train Cont = {avg_train_cont:.4f}, Train SMAPE = {train_smape:.2f}%")

    # ------------------- TESTING -------------------
    model.eval()
    test_mse, test_cont = 0, 0
    test_preds, test_true = [], []

    print(f"🧪 Epoch {epoch + 1}/{num_epochs} - Testing")
    with torch.no_grad():
        for batch_images, batch_texts, batch_prices in tqdm(test_loader, desc="Testing Progress", leave=False):
            pred, text_proj, image_proj = model(batch_images, batch_texts)
            mse = criterion(pred, batch_prices.to(device))
            cont = contrastive_loss(text_proj, image_proj)
            test_mse += mse.item()
            test_cont += cont.item()

            if pred.dim() == 0:
                test_preds.append(pred.cpu().numpy().item())
            else:
                test_preds.extend(pred.cpu().numpy())
            test_true.extend(batch_prices.cpu().numpy())

    avg_test_mse = test_mse / len(test_loader)
    avg_test_cont = test_cont / len(test_loader)
    test_smape = smape(np.array(test_true), np.array(test_preds))
    print(f"📊 Test MSE = {avg_test_mse:.4f}, Test Cont = {avg_test_cont:.4f}, Test SMAPE = {test_smape:.2f}%")

    # --- Save Model Per Epoch ---
    model_save_path = os.path.join(pth, f'multimodal_model_epoch_{epoch + 1}.pth')
    torch.save(model.state_dict(), model_save_path)
    print(f"💾 Model saved for epoch {epoch + 1} at {model_save_path}")

# --- Final Message ---
print(f"\n🏁 Training completed in {time.time() - start_time:.2f} seconds.")


🔹 Epoch 1/100 - Training


Training Progress:  57%|█████▋    | 2133/3750 [4:59:19<4:11:01,  9.31s/it]

In [4]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import ViTImageProcessor, ViTModel, BertTokenizer, BertModel
from tqdm import tqdm
import numpy as np

# --- Custom Dataset and Collate Function ---
def custom_collate_fn(batch):
    """
    Custom collate function to handle PIL Images, strings, and prices.
    Returns images as a list, texts as a list, prices as a tensor.
    """
    images, texts, prices = zip(*batch)
    return list(images), list(texts), torch.tensor(prices, dtype=torch.float32)

class ProductDataset(Dataset):
    def __init__(self, df, image_folder):
        """
        Initialize dataset with DataFrame and image folder.
        - df: Contains [sample_id, product_name, catalog_content_clean, price, value, unit, image]
        - image_folder: Directory with images named in 'image' column (e.g., 33127.jpg)
        """
        self.df = df
        self.image_folder = image_folder
        self.transform = None  # ViTImageProcessor handles preprocessing

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_folder, row['image'])
        try:
            img = Image.open(image_path).convert('RGB')
        except FileNotFoundError:
            print(f"Image not found at {image_path}, using placeholder.")
            img = Image.new('RGB', (224, 224), (0, 0, 0))
        text = row['catalog_content_clean']
        price = float(row['value'])  # Changed 'price' to 'value' (or replace with correct column name)
        return img, text, price

# --- Function to Extract and Save Embeddings ---
def extract_and_save_embeddings(image_model, text_model, image_processor, text_tokenizer, data_loader, save_dir, max_length=128):
    """
    Extract mean-pooled ViT and BERT embeddings and save them with prices and metadata.
    
    Args:
        image_model: ViT model.
        text_model: BERT model.
        image_processor: ViT processor.
        text_tokenizer: BERT tokenizer.
        data_loader: DataLoader providing (images, texts, prices).
        save_dir: Directory to save embeddings (e.g., './embeddings/').
        max_length: Max sequence length for text (default: 128).
    """
    image_model.eval()
    text_model.eval()
    os.makedirs(save_dir, exist_ok=True)
    
    all_image_embeddings = []
    all_text_embeddings = []
    all_prices = []
    all_sample_ids = []
    all_image_paths = []
    
    with torch.no_grad():
        for batch_idx, (batch_images, batch_texts, batch_prices) in enumerate(tqdm(data_loader, desc="Extracting embeddings")):
            # Image embeddings (mean-pooled)
            inputs = image_processor(images=batch_images, return_tensors="pt").to(device)
            image_emb = image_model(**inputs).last_hidden_state.mean(dim=1)  # [batch_size, 768]
            all_image_embeddings.append(image_emb.cpu())
            
            # Text embeddings (mean-pooled)
            inputs = text_tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)
            text_emb = text_model(**inputs).last_hidden_state.mean(dim=1)  # [batch_size, 768]
            all_text_embeddings.append(text_emb.cpu())
            
            # Store prices and metadata
            all_prices.append(batch_prices)
            batch_start_idx = batch_idx * data_loader.batch_size
            batch_end_idx = min(batch_start_idx + len(batch_images), len(data_loader.dataset))
            all_sample_ids.extend(data_loader.dataset.df.iloc[batch_start_idx:batch_end_idx]['sample_id'].tolist())
            all_image_paths.extend([os.path.join(data_loader.dataset.image_folder, data_loader.dataset.df.iloc[i]['image']) 
                                   for i in range(batch_start_idx, batch_end_idx)])

    # Concatenate all
    all_image_embeddings = torch.cat(all_image_embeddings, dim=0)  # [num_samples, 768]
    all_text_embeddings = torch.cat(all_text_embeddings, dim=0)    # [num_samples, 768]
    all_prices = torch.cat(all_prices, dim=0)                      # [num_samples]
    
    # Save embeddings and prices
    torch.save({
        'image_embeddings': all_image_embeddings,
        'text_embeddings': all_text_embeddings,
        'prices': all_prices
    }, os.path.join(save_dir, 'embeddings.pt'))
    
    # Save metadata
    metadata = pd.DataFrame({
        'sample_id': all_sample_ids,
        'image_path': all_image_paths
    })
    metadata.to_csv(os.path.join(save_dir, 'metadata.csv'), index=False)
    
    print(f"✅ Embeddings saved to {save_dir}")
    print(f"Image embeddings shape: {all_image_embeddings.shape} ([num_samples, 768])")
    print(f"Text embeddings shape: {all_text_embeddings.shape} ([num_samples, 768])")
    print(f"Prices shape: {all_prices.shape} ([num_samples])")

# --- Function to Load Saved Embeddings ---
def load_embeddings(save_dir):
    """
    Load saved embeddings, prices, and metadata from disk.
    
    Args:
        save_dir: Directory where embeddings are saved.
    
    Returns:
        image_embeddings: Tensor of shape [num_samples, 768]
        text_embeddings: Tensor of shape [num_samples, 768]
        prices: Tensor of shape [num_samples]
        metadata: DataFrame with sample_id, image_path
    """
    data = torch.load(os.path.join(save_dir, 'embeddings.pt'))
    image_embeddings = data['image_embeddings']
    text_embeddings = data['text_embeddings']
    prices = data['prices']
    metadata = pd.read_csv(os.path.join(save_dir, 'metadata.csv'))
    print(f"Loaded embeddings:")
    print(f"Image embeddings shape: {image_embeddings.shape} ([num_samples, 768])")
    print(f"Text embeddings shape: {text_embeddings.shape} ([num_samples, 768])")
    print(f"Prices shape: {prices.shape} ([num_samples])")
    return image_embeddings, text_embeddings, prices, metadata

# --- Main Script ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load ViT for images
image_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
image_model = ViTModel.from_pretrained('google/vit-base-patch16-224').to(device)
print("ViT model initialized.")

# Load BERT for text
text_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text_model = BertModel.from_pretrained('bert-base-uncased').to(device)
print("BERT model initialized.")

# Define embeddings directory
EMBEDDINGS_DIR = './embeddings/'
os.makedirs(EMBEDDINGS_DIR, exist_ok=True)

# Load data
IMAGE_FOLDER = 'test_images/test_images'
df = pd.read_csv("test/merged_test_with_image.csv")
print(f"Loaded {len(df)} rows")
print("Columns in CSV:", df.columns.tolist())

# Verify required columns
required_columns = ['sample_id', 'catalog_content_clean', 'value', 'image']  # Adjust 'value' to actual price column
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"Missing required columns in CSV: {missing_columns}")

# Split data
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print(f"Train: {len(train_df)} rows, Test: {len(test_df)} rows")

# Create DataLoaders for train and test
try:
    train_dataset = ProductDataset(train_df, IMAGE_FOLDER)
    test_dataset = ProductDataset(test_df, IMAGE_FOLDER)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=True, collate_fn=custom_collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=True, collate_fn=custom_collate_fn)
    print("DataLoaders ready.")
    # Test one batch
    for batch_images, batch_texts, batch_prices in train_loader:
        print(f"Batch loaded: {len(batch_images)} images, {len(batch_texts)} texts, {batch_prices.shape} prices")
        print(f"Image type: {type(batch_images[0])}, Price type: {type(batch_prices)}")
        break
except Exception as e:
    print(f"Error creating DataLoader: {e}")
    raise

# Extract and save embeddings for train and test
extract_and_save_embeddings(
    image_model, text_model, image_processor, text_tokenizer,
    train_loader, os.path.join(EMBEDDINGS_DIR, 'train'), max_length=128
)
extract_and_save_embeddings(
    image_model, text_model, image_processor, text_tokenizer,
    test_loader, os.path.join(EMBEDDINGS_DIR, 'test'), max_length=128
)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViT model initialized.
BERT model initialized.
Loaded 74999 rows
Columns in CSV: ['sample_id', 'product_name', 'catalog_content_clean', 'value', 'unit', 'image']
Train: 59999 rows, Test: 15000 rows
DataLoaders ready.
Batch loaded: 32 images, 32 texts, torch.Size([32]) prices
Image type: <class 'PIL.Image.Image'>, Price type: <class 'torch.Tensor'>


Extracting embeddings:   3%|▎         | 64/1875 [02:42<1:16:31,  2.54s/it]


KeyboardInterrupt: 

In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import ViTImageProcessor, ViTModel, BertTokenizer, BertModel
from tqdm import tqdm
import numpy as np

# --- Custom Dataset and Collate Function ---
def custom_collate_fn(batch):
    """
    Custom collate function to handle PIL Images, strings, and prices.
    Returns images as a list, texts as a list, prices as a tensor.
    """
    images, texts, prices = zip(*batch)
    return list(images), list(texts), torch.tensor(prices, dtype=torch.float32)

class ProductDataset(Dataset):
    def __init__(self, df, image_folder):
        """
        Initialize dataset with DataFrame and image folder.
        - df: Contains [sample_id, product_name, catalog_content_clean, value, unit, image]
        - image_folder: Directory with images named in 'image' column (e.g., 33127.jpg)
        """
        self.df = df
        self.image_folder = image_folder
        self.transform = None  # ViTImageProcessor handles preprocessing

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_folder, row['image'])
        try:
            img = Image.open(image_path).convert('RGB')
        except FileNotFoundError:
            print(f"Image not found at {image_path}, using placeholder.")
            img = Image.new('RGB', (224, 224), (0, 0, 0))
        text = row['catalog_content_clean']
        price = float(row['value'])  # Using 'value' as the price column
        return img, text, price

# --- Function to Extract and Save Embeddings ---
def extract_and_save_embeddings(image_model, text_model, image_processor, text_tokenizer, data_loader, save_dir, max_length=128):
    """
    Extract mean-pooled ViT and BERT embeddings and save them with prices and metadata.
    
    Args:
        image_model: ViT model.
        text_model: BERT model.
        image_processor: ViT processor.
        text_tokenizer: BERT tokenizer.
        data_loader: DataLoader providing (images, texts, prices).
        save_dir: Directory to save embeddings (e.g., './embeddings/train/').
        max_length: Max sequence length for text (default: 128).
    """
    image_model.eval()
    text_model.eval()
    os.makedirs(save_dir, exist_ok=True)
    
    all_image_embeddings = []
    all_text_embeddings = []
    all_prices = []
    all_sample_ids = []
    all_image_paths = []
    
    with torch.no_grad():
        for batch_idx, (batch_images, batch_texts, batch_prices) in enumerate(tqdm(data_loader, desc="Extracting embeddings")):
            # Image embeddings (mean-pooled)
            inputs = image_processor(images=batch_images, return_tensors="pt").to(device)
            image_emb = image_model(**inputs).last_hidden_state.mean(dim=1)  # [batch_size, 768]
            all_image_embeddings.append(image_emb.cpu())
            
            # Text embeddings (mean-pooled)
            inputs = text_tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)
            text_emb = text_model(**inputs).last_hidden_state.mean(dim=1)  # [batch_size, 768]
            all_text_embeddings.append(text_emb.cpu())
            
            # Store prices and metadata
            all_prices.append(batch_prices)
            batch_start_idx = batch_idx * data_loader.batch_size
            batch_end_idx = min(batch_start_idx + len(batch_images), len(data_loader.dataset))
            all_sample_ids.extend(data_loader.dataset.df.iloc[batch_start_idx:batch_end_idx]['sample_id'].tolist())
            all_image_paths.extend([os.path.join(data_loader.dataset.image_folder, data_loader.dataset.df.iloc[i]['image']) 
                                   for i in range(batch_start_idx, batch_end_idx)])

    # Concatenate all
    all_image_embeddings = torch.cat(all_image_embeddings, dim=0)  # [num_samples, 768]
    all_text_embeddings = torch.cat(all_text_embeddings, dim=0)    # [num_samples, 768]
    all_prices = torch.cat(all_prices, dim=0)                      # [num_samples]
    
    # Save embeddings and prices
    torch.save({
        'image_embeddings': all_image_embeddings,
        'text_embeddings': all_text_embeddings,
        'prices': all_prices
    }, os.path.join(save_dir, 'embeddings.pt'))
    
    # Save metadata
    metadata = pd.DataFrame({
        'sample_id': all_sample_ids,
        'image_path': all_image_paths
    })
    metadata.to_csv(os.path.join(save_dir, 'metadata.csv'), index=False)
    
    print(f"✅ Embeddings saved to {save_dir}")
    print(f"Image embeddings shape: {all_image_embeddings.shape} ([num_samples, 768])")
    print(f"Text embeddings shape: {all_text_embeddings.shape} ([num_samples, 768])")
    print(f"Prices shape: {all_prices.shape} ([num_samples])")

# --- Function to Load Saved Embeddings ---
def load_embeddings(save_dir):
    """
    Load saved embeddings, prices, and metadata from disk.
    
    Args:
        save_dir: Directory where embeddings are saved.
    
    Returns:
        image_embeddings: Tensor of shape [num_samples, 768]
        text_embeddings: Tensor of shape [num_samples, 768]
        prices: Tensor of shape [num_samples]
        metadata: DataFrame with sample_id, image_path
    """
    data = torch.load(os.path.join(save_dir, 'embeddings.pt'))
    image_embeddings = data['image_embeddings']
    text_embeddings = data['text_embeddings']
    prices = data['prices']
    metadata = pd.read_csv(os.path.join(save_dir, 'metadata.csv'))
    print(f"Loaded embeddings:")
    print(f"Image embeddings shape: {image_embeddings.shape} ([num_samples, 768])")
    print(f"Text embeddings shape: {text_embeddings.shape} ([num_samples, 768])")
    print(f"Prices shape: {prices.shape} ([num_samples])")
    return image_embeddings, text_embeddings, prices, metadata

# --- Main Script ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load ViT for images
image_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
image_model = ViTModel.from_pretrained('google/vit-base-patch16-224').to(device)
print("ViT model initialized.")

# Load BERT for text
text_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text_model = BertModel.from_pretrained('bert-base-uncased').to(device)
print("BERT model initialized.")

# Define embeddings directory
EMBEDDINGS_DIR = './embeddings/train/'
os.makedirs(EMBEDDINGS_DIR, exist_ok=True)

# Load data
IMAGE_FOLDER = 'test_images/test_images'  # Assuming this is your training image folder
df = pd.read_csv("test/merged_test_with_image.csv")  # Assuming this is your training CSV
print(f"Loaded {len(df)} rows")
print("Columns in CSV:", df.columns.tolist())

# Verify required columns
required_columns = ['sample_id', 'catalog_content_clean', 'value', 'image']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"Missing required columns in CSV: {missing_columns}")

# Create DataLoader for training data only
try:
    train_dataset = ProductDataset(df, IMAGE_FOLDER)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=True, collate_fn=custom_collate_fn)
    print("DataLoader ready for training data.")
    # Test one batch
    for batch_images, batch_texts, batch_prices in train_loader:
        print(f"Batch loaded: {len(batch_images)} images, {len(batch_texts)} texts, {batch_prices.shape} prices")
        print(f"Image type: {type(batch_images[0])}, Price type: {type(batch_prices)}")
        break
except Exception as e:
    print(f"Error creating DataLoader: {e}")
    raise

# Extract and save embeddings for training data only
extract_and_save_embeddings(
    image_model, text_model, image_processor, text_tokenizer,
    train_loader, EMBEDDINGS_DIR, max_length=128
)